# [Advent of Code 2021](https://adventofcode.com/2021)

Partially inspired by [Norvig's pytudes](https://github.com/norvig/pytudes/blob/main/ipynb/Advent-2020.ipynb), although I probably won't be using the same style.

In [38]:
# Imports

from typing import Dict, List, Iterable, Callable, TypeVar

T = TypeVar('T')

In [29]:
# Helper functions

def data(day: int, parser: Callable[[str], T]) -> List[T]:
    with open(f"../data/2021/day{day}.txt") as f:
        return [parser(line.strip()) for line in f.readlines()]

## Day 1

Determine number of increments in list

In [46]:
data1 = data(1, int)

In [47]:
def day1(data: List[int]) -> int:
  count, current = 0, data[0]
  for i in data[1:]:
    if i > current:
      count += 1
    current = i
  return count

day1(data1)

1688

In [48]:
def win3(data: List[int]) -> List[int]:
  for i in range(len(data) - 2):
    yield sum(data[i:i+3])

day1(list(win3(data1)))

1728

## Day 2

In [44]:
# Scratch